In [ ]:
#import libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score,make_scorer,average_precision_score,f1_score,recall_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import shap

In [ ]:
#Load data
df = pd.read_csv('/kaggle/input/telco-customer-churn-11-1-3/telco.csv')
df.head()

In [ ]:
# Check the size of the dataframe
print(f"Shape :{df.shape}")
# Check the columns
print(f"Columns: {df.columns}")

In [ ]:
# We need to drop the following columns since they are highly correlated to target and should not be used for prediction
#found high correlation  between target var and satisfaction score later so removing it as well
df.drop(['Churn Score', 'CLTV','Churn Category','Churn Reason','Satisfaction Score', 'Customer Status' ],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['Country'].unique()

In [ ]:
df['State'].unique()

In [ ]:
df['City'].unique()

In [ ]:
df['Payment Method'].unique()

In [ ]:
#Removing unnecessary columns which do not have any info
df.drop(['Customer ID','Country','State','Quarter'],axis=1,inplace=True)
df.head()

In [ ]:
 # Remove whitespaces from city column
df['Payment Method'].replace(' ','',regex=True,inplace=True)
df.head()

In [ ]:
df.columns = df.columns.str.replace(' ','_')
df.head()

In [ ]:
df.replace(' ', '_', regex=True, inplace=True)

**Identifying missing data**

In [ ]:
full_description = df.describe(include='all').T
full_description

In [ ]:
missing_data = df[df['Offer'].isnull()]
missing_data

**EDA**

In [ ]:
full_data = df.groupby(['Contract','Under_30','Churn_Label'])[['Gender']].count()
full_data

In [ ]:
missing_data = df[(df['Offer'].isnull()) ]
missing_data2 = missing_data.groupby(['Under_30','Contract'])[['Gender']].count()

In [ ]:
missing_data2

**Handling missing values and creating rich features**

XGBoost can handle missing values be default

In [ ]:
# Define the custom labels for missing values
OLDER_MISSING_LABEL = 'Offer_Missing_Age_30_Plus'
YOUNGER_MISSING_LABEL = 'Offer_Missing_Age_Under_30'
    
# Create the new imputed column, starting with the original values
df['Offer_Imputed'] = df['Offer']

# Fill NaN values based on the age condition
# For NaN rows: if Age >= 30, use the older label; otherwise, use the younger label.

# Identify rows where Offer is currently missing (NaN)
missing_mask = df['Offer_Imputed'].isna()

# Apply the older label imputation
df.loc[(df['Age'] >= 30) & missing_mask, 'Offer_Imputed'] = OLDER_MISSING_LABEL

# Apply the younger label imputation (Age < 30)
df.loc[(df['Age'] < 30) & missing_mask, 'Offer_Imputed'] = YOUNGER_MISSING_LABEL

# Finally, convert the new imputed column to the 'category' dtype
# df['Offer_Imputed'] = df['Offer_Imputed'].astype('category')

In [ ]:
for col in df.columns:
    print(f"Column {col}: {df[col].dtype},{df[col].unique()}")
    

In [ ]:
# converting object to category to avoid one-hot encoding
categorical_cols = df.select_dtypes(include=['object']).columns

# Convert the dtype
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
# Data used to make predictions
X = df.drop(['Churn_Label'],axis=1)
X.head()

In [ ]:
# Data to predict.
# y = df['Churn_Label']
df['Churn_val'] = np.where(df['Churn_Label'].str.lower() == 'yes', 1, 0)
y= df['Churn_val']
y.head()

**One hot encoding not required in modern xgboost**

In [ ]:
# X_encoded = pd.get_dummies(X, columns=[
#     'City', 'Gender', 'Senior_Citizen', 'Married', 'Dependents',
#     'Phone_Service', 'Multiple_Lines', 'Internet_Service', 'Online_Security',
#     'Online_Backup', 'Device_Protection_Plan', 'Premium_Tech_Support', 'Streaming_TV',
#     'Streaming_Movies', 'Contract', 'Paperless_Billing', 'Payment_Method'
# ], dtype='int')
# X_encoded.head()

**Building a preliminary model**

In [ ]:
sum(y)/len(y)

In [ ]:
#Creating train/test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
sum(y_test)/len(y_test)

In [ ]:
print(f"Training Size: {len(X_train)}")
print(f"Test Size: {len(X_test)}")
print(f'Training Set Shape: {X_train.shape}')

In [ ]:
print(f"Training Size: {len(y_train)}")
print(f"Test Size: {len(y_test)}")
print(f'Training Set Shape: {y_train.shape}')

Using XGBoost

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', seed=42,enable_categorical=True) # to handle categorical values like object data type
clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric='aucpr', eval_set=[(X_test, y_test)])

In [ ]:
# 3. METHOD A: XGBoost Built-in Feature Importance
print("--- Method A: Built-in Feature Importance (Gain) ---")
# 'Gain' is the most popular metric; it measures the improvement in accuracy 
# brought by a feature to the branches it is on.

fig, ax = plt.subplots(figsize=(10, 6))
xgb.plot_importance(clf_xgb, importance_type='gain', max_num_features=10, ax=ax, title='XGBoost Feature Importance (Gain)')
plt.show()

# You can also extract the raw scores into a DataFrame:
importance = clf_xgb.get_booster().get_score(importance_type='gain')
importance_df = pd.DataFrame(
    list(importance.items()), 
    columns=['Feature', 'Importance_Gain']
).sort_values(by='Importance_Gain', ascending=False)
print("Top 5 Features by Gain:")
print(importance_df.head())

In [ ]:
# # Assuming your DataFrame is named 'df'
# # Replace 'Suspicious_Feature' with the actual column name that has the gain of 229

# leak_correlation = df['Satisfaction_Score'].corr(df['Churn_val'])

# print(f"Correlation between the Suspicious Feature and Target: {leak_correlation:.4f}")

In [ ]:
# print(pd.crosstab(df['Satisfaction_Score'], df['Churn_val'], normalize='index'))

In [ ]:
# # 4. METHOD B: SHAP (SHapley Additive exPlanations) for Deeper Insight
# print("\n--- Method B: SHAP Values for Model Interpretation ---")
# # SHAP provides more reliable feature importance and explains individual predictions.

# # Create SHAP Explainer
# explainer = shap.TreeExplainer(clf_xgb)
# # Calculate SHAP values for the test set
# shap_values = explainer.shap_values(X_test)

# # Plot 1: SHAP Summary Plot (Global Feature Importance)
# # This plot shows the average magnitude of SHAP values for each feature.
# # It is often considered more stable and reliable than the built-in XGBoost importance.
# print("\nSHAP Summary Plot (Global Feature Ranking):")
# shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
# plt.title("SHAP Global Feature Importance")
# plt.show()

# # Plot 2: SHAP Decision Plot (Impact Visualization)
# # This plot shows the distribution of SHAP values for each feature.
# # Color represents the feature value (e.g., high value = red, low value = blue).
# print("SHAP Summary Plot (Feature Impact & Direction):")
# shap.summary_plot(shap_values, X_test, show=False)
# plt.show()

In [ ]:
y_pred = clf_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=clf_xgb.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Did not leave', 'Left'])
disp.plot()
plt.show()

# Optimizing Parameters using Cross Validation and Grid Search()

In [ ]:
scale_pos_weight_init_val = (len(y_train)- sum(y_train))/sum(y_train)
scale_pos_weight_init_val

In [ ]:
# param_grid ={
#     'max_depth': [4],
#     'learning_rate' : [0.1],
#     'gamma': [0.25],
#     'reg_lambda': [0,1.0, 10.0],
#     'scale_pos_weight': [1,2,2.5,3.0,3.5 ,5] 
# }

# # 2. INITIALIZE MODEL AND CROSS-VALIDATION STRATEGY
# # Use StratifiedKFold for imbalanced data to ensure each fold has the same proportion of the target class.
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# #3. SET UP GRID SEARCH
# # We use 'roc_auc' as the scoring metric, which is standard for binary classification problems.
# grid_search = GridSearchCV(
#     estimator=clf_xgb,
#     param_grid=param_grid,
#     scoring='recall',         # Metric to optimize (AUC-ROC)
#     cv=cv,                     # Use Stratified K-Fold
#     verbose=3,                 # Controls the verbosity of the output
#     n_jobs=-1                  # Use all CPU cores for parallel processing
# )

# # 4. RUN THE SEARCH
# print("\n--- Starting Grid Search Cross-Validation ---")
# # 
# grid_search.fit(X_train, y_train)

# # 5. DISPLAY RESULTS
# print("\n--- Grid Search Results ---")

# # Best parameters found
# print(f"Best Parameters: {grid_search.best_params_}")

# # Best cross-validation score achieved
# print(f"Best CV AUC-PR Score: {grid_search.best_score_:.4f}")

# # Retrieve the best model
# best_xgb_model = grid_search.best_estimator_

# # Final evaluation on the unseen test set (crucial!)
# y_test_pred_proba = best_xgb_model.predict_proba(X_test)[:, 1]
# final_auc = roc_auc_score(y_test, y_test_pred_proba)

# print(f"\nFinal Test Set AUC-PR Score: {final_auc:.4f}")

In [ ]:
# Best Parameters: {'gamma': 0.25, 'learning_rate': 0.1, 'max_depth': 4, 'reg_lambda': 0, 'scale_pos_weight': 1}
# Best CV ROC-AUC Score: 0.9132

# Final Test Set ROC-AUC Score: 0.9115

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
seed=42,
gamma=0.25,
learning_rate=0.1,
max_depth=4,
reg_lambda=0,
scale_pos_weight=3.0,
subsample=0.9,
colsample_bytree=0.5,
n_estimators=200,
 enable_categorical =True)
clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric='aucpr', eval_set=[(X_test, y_test)])

In [ ]:
y_pred = clf_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=clf_xgb.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Did not leave', 'Left'])
disp.plot()
plt.savefig('confusion_matrix.png')
plt.show()

**Drawing an XGBoost Tree**

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
seed=42,
gamma=0.25,
learning_rate=0.1,
max_depth=4,
reg_lambda=10,
scale_pos_weight=3,
subsample=0.9,
colsample_bytree=0.5,
n_estimators=2,
                           enable_categorical=True)
clf_xgb.fit(X_train, y_train)

# weight is the number of times a feature is used to split the data across all trees.(no. of times a features is used in a branch or root across all trees)
# gain is the average gain across all splits the feature is used in.
# cover is the average coverage across all splits the feature is used in.
# total_gain is the total gain across all splits the feature is used in.
# total_cover is the total coverage across all splits the feature is used in.

bst = clf_xgb.get_booster()
for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
    print('%s: ' % importance_type, bst.get_score(importance_type=importance_type))
node_params = {'shape': 'box',
               'style': 'filled, rounded',
               'fillcolor': '#78cbe'}
leaf_params = {'shape': 'box', 'style': 'filled',
               'fillcolor': '#e48038'}

xgb.to_graphviz(clf_xgb, num_trees=1, size="10,10",
                condition_node_params=node_params,
                leaf_node_params=leaf_params)